In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.tsa.seasonal import seasonal_decompose 
from statsmodels.graphics.tsaplots import plot_acf

from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras import Input

import math

In [2]:
df = pd.read_csv('df_nivel_rios.csv')
df = df.iloc[:, 1:]

# Converter 'Data' para datetime
df['Data'] = pd.to_datetime(df['Data'])

df = df.set_index('Data')
df = df[['Nível Guaíba (cm)']]

split_ratio = 0.8
split_index = int(len(df) * split_ratio)

train = df[:split_index]
test = df[split_index:]

scaler = MinMaxScaler()
scaler.fit(train)
train_data = scaler.transform(train)
test_data = scaler.transform(test)

def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# Define sequence length
seq_length = 720    # 1 mês

# Create sequences for training set
X_train, y_train = create_sequences(train_data, seq_length)

# Create sequences for testing set
X_test, y_test = create_sequences(test_data, seq_length)

# Reshape input data
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

model = Sequential([
    Input(shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(units=64, return_sequences=True),
    #LSTM(units=32, return_sequences=True),
    LSTM(units=16, return_sequences=True),
    Dense(units=1)
])
model.compile(loss='mean_squared_error', optimizer='adam')

model.fit(X_train, y_train, epochs=2, batch_size=1, verbose=1)


Epoch 1/2
44372/44372 ━━━━━━━━━━━━━━━━━━━━ 6391s 144ms/step - loss: 0.0123
Epoch 2/2
44372/44372 ━━━━━━━━━━━━━━━━━━━━ 6370s 144ms/step - loss: 0.0113


In [3]:
trainScore = model.evaluate(X_train, y_train, verbose=0)
print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore, math.sqrt(trainScore)))
testScore = model.evaluate(X_test, y_test, verbose=0)
print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore, math.sqrt(testScore)))

Train Score: 0.01 MSE (0.10 RMSE)
Test Score: 0.04 MSE (0.19 RMSE)


In [ ]:
model.export('Modelo_LSTM')

In [4]:
#model = Sequential()
#model.add(LSTM(200, activation= "relu" , input_shape = (n_input , n_features)))
#model.add(Dense(1))
#model.compile(optimizer = "adam" , loss="mse")

In [5]:
#plt.plot(df)
#plt.show()

In [6]:
#results = seasonal_decompose(df)
#results.seasonal[-5000:].plot()

In [7]:
#plot_acf(df)
#plt.show();

In [8]:
train = df[:-48]
train

,Nível Guaíba (cm)
Data,
2017-11-14 12:00:00,114.0
2017-11-14 13:00:00,114.0
2017-11-14 14:00:00,114.0
2017-11-14 15:00:00,114.0
2017-11-14 16:00:00,114.0
...,...
2024-04-17 21:00:00,98.0
2024-04-17 22:00:00,97.0
2024-04-17 23:00:00,96.0


In [9]:
df

,Nível Guaíba (cm)
Data,
2017-11-14 12:00:00,114.0
2017-11-14 13:00:00,114.0
2017-11-14 14:00:00,114.0
2017-11-14 15:00:00,114.0
2017-11-14 16:00:00,114.0
...,...
2024-04-19 21:00:00,104.0
2024-04-19 22:00:00,104.0
2024-04-19 23:00:00,104.0
